<a href="https://colab.research.google.com/github/bobcoi03/learnAI/blob/main/learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import nn

In [5]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [13]:
batch_size = 32

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N C H W]: {X.shape}")
  print(f"Shape of y: {y.shape}")
  break

Shape of X [N C H W]: torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32])


In [3]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device}")

Using cuda


In [4]:
class NeuralNetwork(nn.Module):
        def __init__(self):
            super().__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(28*28, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 10)
            )

        def forward(self, x):
          x = self.flatten(x)
          logits = self.linear_relu_stack(x)
          return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
# loss function & optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [11]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    # for each batch
    X, y = X.to(device), y.to(device)

    # prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n---------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------------------
loss: 1.051989  [    0/ 1875]
loss: 1.082171  [ 3200/ 1875]
loss: 1.071012  [ 6400/ 1875]
loss: 1.170692  [ 9600/ 1875]
loss: 0.979014  [12800/ 1875]
loss: 1.187739  [16000/ 1875]
loss: 0.880234  [19200/ 1875]
loss: 0.892434  [22400/ 1875]
loss: 0.976151  [25600/ 1875]
loss: 1.076916  [28800/ 1875]
loss: 0.907254  [32000/ 1875]
loss: 0.831055  [35200/ 1875]
loss: 1.142928  [38400/ 1875]
loss: 0.988205  [41600/ 1875]
loss: 0.917190  [44800/ 1875]
loss: 0.819411  [48000/ 1875]
loss: 0.944311  [51200/ 1875]
loss: 1.079031  [54400/ 1875]
loss: 0.817912  [57600/ 1875]
Test Error: 
 Accuracy: 67.4%, Avg loss: 0.902495 

Epoch 2
---------------------
loss: 0.838494  [    0/ 1875]
loss: 0.922125  [ 3200/ 1875]
loss: 0.891896  [ 6400/ 1875]
loss: 1.032102  [ 9600/ 1875]
loss: 0.799471  [12800/ 1875]
loss: 1.046920  [16000/ 1875]
loss: 0.714473  [19200/ 1875]
loss: 0.713199  [22400/ 1875]
loss: 0.825683  [25600/ 1875]
loss: 1.001240  [28800/ 1875]
loss: 0.785582  [3

In [16]:
# save model
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
